In [112]:
import torch
import torch.nn.functional as F
from torch import nn  # for

import numpy as np  # for
import pandas as pd  # for


import plotly.graph_objects as go
import plotly.express as px
from plotly import subplots

from tqdm.notebook import tqdm  # for

import random  # for

In [113]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("cuda available")
else:
    device = torch.device("cpu")

<div dir="rtl" lang="he" xml:lang="he">


# מודלים יוצרים - _gan models_
ניתן להשתמש ברשתות נויירונים גם ליצירה של מידע הנראה כאמיתי. 
נבנה מודל כזה שיוכל ליצור כתב יד בצורה אמינה.

<div dir="rtl" lang="he" xml:lang="he">

### יבוא הדאטא
נוריד את הקבצים מאתר האינטרנט בצורה ידנית, ונלחץ אותם במקום הנכון.
הדאטא מגיע כקובץ CSV טבלה, כך שהעמודה הראשונה היא התגית _label_ וכל שאר העמודות הם העמודות של התמונה


נשתמש בפונקצייה `load_text` בשביל לטעון את הקבצים, 
לאחר מכן נמיר את הקבצים לוקטור.

In [114]:
# script to download files
##!!! only for files that downloaded at `.tar.gz` format


import requests  # for downloading files
import tarfile  # for extracting files
import pathlib  # check if file exists
import zipfile

# URL = "https://www.kaggle.com/datasets/hojjatk/mnist-dataset/download?datasetVersionNumber=1"
URL = "https://www.kaggle.com/datasets/oddrationale/mnist-in-csv/download?datasetVersionNumber=2"
DATA_FOLDER = "data"
MODEL_NAME = "mnist"
data_folder = pathlib.Path(f"./{DATA_FOLDER}")
data_set_folder = pathlib.Path(f"{DATA_FOLDER}/{MODEL_NAME}")
downloaded_file = pathlib.Path(f"{DATA_FOLDER}/{MODEL_NAME}/archve.zip")


if not data_folder.exists():
    data_folder.mkdir()
if not data_set_folder.exists():
    data_set_folder.mkdir()
    response = requests.get(URL)
    open(downloaded_file, "wb").write(response.content)
if downloaded_file.exists():
    # extracting file
    with zipfile.ZipFile(downloaded_file, "r") as zip_ref:
        zip_ref.extractall(data_set_folder)
    # delet downloaded file
    if downloaded_file.exists():
        if input("do you want to delete the downloaded file?\ny or n") in "Yy":
            downloaded_file.unlink()

In [115]:
vec = np.genfromtxt("data/mnist/mnist_test.csv", delimiter=",", skip_header=1)
print(
    f"""
vec shape: {vec.shape}
vec dtpye: {vec.dtype}
vec head: 
{vec[0:5]}
    """
)


vec shape: (10000, 785)
vec dtpye: float64
vec head: 
[[7. 0. 0. ... 0. 0. 0.]
 [2. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [4. 0. 0. ... 0. 0. 0.]]
    


<div dir="rtl" lang="he" xml:lang="he">

העמודה הראשונה היא התוית _label_ 
של התמונה, וכל שאר 784 העמודות זה התמונה בשחור לבן. 

In [116]:
labels = torch.from_numpy(vec[:, 0]).reshape(len(vec), 1).to(dtype=torch.float32)
img = torch.from_numpy(vec[:, 1:]).reshape(10000, 28, 28).to(dtype=torch.float32)
data_set = torch.utils.data.TensorDataset(img, labels)

In [117]:
dataloader = torch.utils.data.DataLoader(data_set, batch_size=11, shuffle=True)

In [118]:
next(iter(dataloader.batch_sampler))

[4748, 1577, 7508, 1658, 7943, 8531, 5605, 6136, 7284, 7470, 6960]

In [119]:
next(iter(dataloader.batch_sampler))


[1579, 5582, 2782, 531, 7129, 4509, 9910, 1785, 6533, 9298, 4382]

<div dir="rtl" lang="he" xml:lang="he">


### הצגת המספרים
בפונקציה  
`imshow` 
של 
_plotly_ 
ניתן להציג סדרת תמונות


In [120]:
fig = px.imshow(
    data_set[0:10][0], facet_col=0, facet_col_wrap=4, labels={"facet_col": "index"}
)
fig.show()

<div dir="rtl" lang="he" xml:lang="he">

## מודל מעריך
נבנה את המודל שיעריך האם התמונות שניתנות לו הם אמיתיות או לא


מבנה הרשת הוא פשוט: 
יש שלוש שכבות ליניאריות 
_dence_
עם פונקציית אקטיבציה 
_ReLU_
לאחר כל שכבה כזאת מורידים חלק מהוקטורים עם 
_dropout_ 

שפר לשים לב שהשכבות הלינאירות בהתחלה מגדילים את התמונה ורק לאחר מקטינים ומחליטים מה תהיה התמונה. 

In [121]:
class Discriminator(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(784, 1024),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        x = x.view(x.size(0), 784)
        output = self.model(x)
        return output

In [122]:
discriminator = Discriminator()
batch = data_set[:10][0]
discriminator(batch)

tensor([[6.5535e-01],
        [3.6485e-01],
        [1.9473e-03],
        [9.9984e-01],
        [5.7849e-02],
        [9.4425e-01],
        [3.8371e-01],
        [3.1504e-03],
        [4.0831e-04],
        [8.1989e-01]], grad_fn=<SigmoidBackward0>)

<div dir="rtl" lang="he" xml:lang="he">

## הגנרטור _Generator_

הגנרטור הוא מודל שתפקידו ליצור תמונות יש מאין. 
 הוא עושה זאת ע"י מודל שהוא מעין ההופכי של המודל המחליט - ה
 _Discriminator_. 
 הוא מתחיל מוקטור בגודל 100 אותו הוא מקבל כארגומנט
 (
 ע"פ רוב זה יהיה וקטור אקראי המתפלג אחיד
 )


In [123]:
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(100, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Linear(1024, 784),
            nn.Tanh(),
        )

    def forward(self, x):
        output = self.model(x)
        output = output.view(x.size(0), 28, 28)
        return output

In [124]:
generator = Generator()
batch = torch.randn(10, 100)
px.imshow(
    generator(batch).detach(),
    facet_col=0,
    facet_col_wrap=4,
    color_continuous_scale="gray",
)

<div dir="rtl" lang="he" xml:lang="he">

# אימון המודל
נבנה  `Trainer` בשביל לאמן את המודלים שלנו
הוא יכיל: 
- שני מודלים , את המודל היוצר 
`Generator`
ואת המודל המבקר
`Discriminator`
- לכל אחד מהמודלים נגדיר אופטימייזר שונה

### תהליך האימון 

לכל קבוצהFIXIT נעשה את הדברים הבאים:
ניצור קבוצת תמונות פייק (מתוך התפלגות נורמלית) ונאמן את המודל השופטFIXIT
להבחין בן התמונות. 
- ניתן לגנרטור ליצור כמה תמונות ונראה אם האלגוריתם השופט מזהה שהם לא תמונות אמיתיות

In [125]:
from typing import Union, Type, Callable, Optional, Any, List, Tuple

ModelType = nn.Module
TensorType = torch.Tensor
OptimazerType = Type[torch.optim.Optimizer]
LossFunctoinType = Callable[[TensorType, TensorType], TensorType]
SchedulerType = Type[torch.optim.lr_scheduler.LRScheduler]
VisulizeFuntoinType = Callable[[float, Optional[Any]], None]
DataType = Tuple[torch.Tensor, torch.Tensor]
PathType = str
FileType = str

In [126]:
import math  # for cilo
from tqdm.notebook import tqdm

DATA = 0
LABELS = 1


class TrainerGan:
    def __init__(
        self,
        discriminator: ModelType,
        generator: ModelType,
        loss_function: LossFunctoinType,
        optimizer_discriminator: OptimazerType,
        optimizer_generator: OptimazerType,
        scheduler: Optional[SchedulerType] = None,
    ):
        self.discriminator = discriminator
        self.generator = generator
        self.loss_function = loss_function
        self.optimizer_discriminator = optimizer_discriminator
        self.optimizer_generator = optimizer_generator
        self.scheduler = scheduler
        self.loss_list_D = []
        self.loss_list_G = []
        self.data = None

    def train(
        self,
        data_set: DataType,
        batch_size: int,
        times: int = 1,
        visualization: int = 10,
    ):
        """train model"""
        data_loader = torch.utils.data.DataLoader(data_set, batch_size=batch_size)
        # creat figure to disply the loss funtion
        for epoch in range(times):
            for indexes in tqdm(data_loader.batch_sampler):
                # Data for training the discriminator
                real_X, real_y = data_set[indexes]
                latent_space_samples = torch.randn(batch_size, 100)

                # generate numbers
                self.generator.eval()  # set generator to eval mode to avoid training it
                generated_X = self.generator.forward(latent_space_samples)
                generated_y = torch.zeros(batch_size, 1)
                all_samples = torch.cat((real_X, generated_X))
                all_labels = torch.cat((real_y, generated_y))

                # Training the discriminator
                self.discriminator.train()
                self.discriminator.zero_grad()
                output_discriminator = self.discriminator.forward(all_samples)
                loss_D = self.loss_function(output_discriminator, all_labels)
                loss_D.backward()
                self.optimizer_discriminator.step()

                # Data for training the generator
                latent_space_samples = torch.randn((batch_size, 100))

                self.generator.train()
                self.generator.zero_grad()
                generated_X = self.generator(latent_space_samples)
                self.discriminator.eval()
                output_discriminator_generated = self.discriminator(generated_X)
                loss_G = self.loss_function(
                    output_discriminator_generated, torch.ones(batch_size, 1)
                )
                loss_G.backward()
                self.optimizer_generator.step()

```python
trainer = Triner(model,loss,optim,sch,visul)
trainer.set_plain("first plain", )
trainer.train(data,10,shappe=Treu,lr=0.1,gamma:float=None,schler_gama:str="",plean:str="")
```

In [127]:
discriminator = Discriminator().to(device)
generator = Generator().to(device)

In [128]:
lr = 0.0001
num_epochs = 50
loss_function = nn.BCELoss()

optimizer_discriminator = torch.optim.Adam(discriminator.parameters(), lr=lr)
optimizer_generator = torch.optim.Adam(generator.parameters(), lr=lr)

In [129]:
trainer = TrainerGan(
    discriminator,
    generator,
    loss_function,
    optimizer_discriminator,
    optimizer_generator,
)

In [130]:
trainer.train(data_set, batch_size=128, times=1)


  0%|          | 0/79 [00:00<?, ?it/s]

In [131]:
out_gen = trainer.generator(torch.randn(10, 100))
predict = trainer.discriminator(out_gen)
px.imshow(
    out_gen.detach(), facet_col=0, facet_col_wrap=4, color_continuous_scale="gray"
).show()
print(predict)

tensor([[0.4978],
        [0.4987],
        [0.4977],
        [0.4984],
        [0.4968],
        [0.4976],
        [0.4987],
        [0.4979],
        [0.4977],
        [0.4980]], grad_fn=<SigmoidBackward0>)


In [132]:
from winsound import Beep
from IPython.display import clear_output
batch_size = [128, 64, 32, 16, 8, 4, 2, 1]

for _ in range(10):
    random_batch_size = random.choice(batch_size)
    trainer.train(data_set, random_batch_size)
    batch = torch.randn(10, 100)
    generator.eval()
    out = generator(batch)
    all_out = torch.cat((all_out, out))
    im = all_out.detach().reshape(-1, 10, 28, 28)
    clear_output(wait=True)
    px.imshow(
        im,
        animation_frame=0,
        facet_col=1,
        facet_col_wrap=4,
        labels={"animation_frame": "round"},
    ).show()
    Beep(400, 250)

  0%|          | 0/5000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
import pathlib

DATA_FOLDER = "data"
MODEL_NAME = "mnist"

folder_path = pathlib.Path(f"{DATA_FOLDER}/{MODEL_NAME}/trained_models")
model_path = pathlib.Path(
    f"{DATA_FOLDER}/{MODEL_NAME}/trained_models/{MODEL_NAME}_gan_G_0604.pth"
)
if input("do you want save module on disc?") in "yY":
    if not folder_path.exists():
        folder_path.mkdir()

    torch.save(generator.state_dict(), model_path)